# Homework 08: Exploratory Data Analysis (EDA)

**Assignment**: Create a comprehensive EDA notebook for financial dataset analysis.

## Objectives
- Statistical summaries and data profiling
- Distributional analysis with visualizations
- Bivariate relationship exploration
- Document findings, risks, and assumptions

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import utils
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Homework 08: Exploratory Data Analysis")

## 1. Data Loading and Initial Inspection

In [ ]:
# Load financial dataset for EDA
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JPM', 'JNJ']
print(f"Loading data for portfolio: {symbols}")

# Fetch comprehensive dataset
raw_data = utils.fetch_multiple_stocks(symbols, prefer_alphavantage=False, period='2y')

if not raw_data.empty:
    # Engineer additional features for EDA
    enhanced_data = []
    
    for symbol in symbols:
        symbol_data = raw_data[raw_data['symbol'] == symbol].copy()
        symbol_data = symbol_data.sort_values('date')
        
        # Price-based features
        symbol_data['daily_return'] = symbol_data['close'].pct_change()
        symbol_data['log_return'] = np.log(symbol_data['close'] / symbol_data['close'].shift(1))
        symbol_data['price_range'] = symbol_data['high'] - symbol_data['low']
        symbol_data['price_range_pct'] = symbol_data['price_range'] / symbol_data['close']
        
        # Volume features
        symbol_data['volume_ma_20'] = symbol_data['volume'].rolling(20).mean()
        symbol_data['volume_ratio'] = symbol_data['volume'] / symbol_data['volume_ma_20']
        
        # Technical indicators
        symbol_data['sma_20'] = symbol_data['close'].rolling(20).mean()
        symbol_data['sma_50'] = symbol_data['close'].rolling(50).mean()
        symbol_data['price_to_sma20'] = symbol_data['close'] / symbol_data['sma_20']
        
        # Volatility
        symbol_data['volatility_20'] = symbol_data['daily_return'].rolling(20).std()
        
        # Time features
        symbol_data['year'] = symbol_data['date'].dt.year
        symbol_data['month'] = symbol_data['date'].dt.month
        symbol_data['weekday'] = symbol_data['date'].dt.weekday
        symbol_data['quarter'] = symbol_data['date'].dt.quarter
        
        enhanced_data.append(symbol_data)
    
    df = pd.concat(enhanced_data, ignore_index=True)
    df = df.dropna()
    
    print(f"✅ Dataset loaded and enhanced: {df.shape}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(f"Symbols: {df['symbol'].unique()}")
else:
    print("❌ Failed to load data")

## 2. Statistical Summaries and Data Profiling

In [ ]:
if not df.empty:
    print("📋 Dataset Information:")
    print(df.info())
    
    print("\n📊 Descriptive Statistics:")
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    print(df[numeric_cols].describe())
    
    print("\n❓ Missing Value Analysis:")
    missing_counts = df.isnull().sum()
    missing_pct = (missing_counts / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_counts,
        'Missing %': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))
    
    print("\n🏷️ Categorical Variables:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        print(f"{col}: {df[col].nunique()} unique values")
        print(f"  Values: {df[col].unique()[:10]}")
    
    print("\n📈 Key Numeric Variables Summary:")
    key_vars = ['close', 'volume', 'daily_return', 'volatility_20', 'price_range_pct']
    for var in key_vars:
        if var in df.columns:
            print(f"{var}:")
            print(f"  Range: {df[var].min():.4f} to {df[var].max():.4f}")
            print(f"  Mean: {df[var].mean():.4f}, Median: {df[var].median():.4f}")
            print(f"  Skewness: {df[var].skew():.4f}, Kurtosis: {df[var].kurtosis():.4f}")

## 3. Distributional Analysis

In [ ]:
if not df.empty:
    # Distribution plots for key variables
    fig, axes = plt.subplots(3, 2, figsize=(15, 12))
    
    # Daily Returns Distribution
    axes[0,0].hist(df['daily_return'], bins=50, alpha=0.7, edgecolor='black')
    axes[0,0].axvline(df['daily_return'].mean(), color='red', linestyle='--', label=f'Mean: {df["daily_return"].mean():.4f}')
    axes[0,0].axvline(df['daily_return'].median(), color='green', linestyle='--', label=f'Median: {df["daily_return"].median():.4f}')
    axes[0,0].set_title('Daily Returns Distribution')
    axes[0,0].set_xlabel('Daily Return')
    axes[0,0].legend()
    
    # Volume Distribution (log scale)
    axes[0,1].hist(np.log10(df['volume']), bins=50, alpha=0.7, edgecolor='black')
    axes[0,1].set_title('Volume Distribution (Log10 Scale)')
    axes[0,1].set_xlabel('Log10(Volume)')
    
    # Volatility Distribution
    axes[1,0].hist(df['volatility_20'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    axes[1,0].set_title('20-Day Volatility Distribution')
    axes[1,0].set_xlabel('Volatility')
    
    # Price Range Percentage
    axes[1,1].hist(df['price_range_pct'], bins=50, alpha=0.7, edgecolor='black')
    axes[1,1].set_title('Daily Price Range (% of Close)')
    axes[1,1].set_xlabel('Price Range %')
    
    # Box plots by symbol
    df.boxplot(column='daily_return', by='symbol', ax=axes[2,0])
    axes[2,0].set_title('Daily Returns by Symbol')
    axes[2,0].set_xlabel('Symbol')
    
    # Volume ratio distribution
    axes[2,1].hist(df['volume_ratio'].dropna(), bins=50, alpha=0.7, edgecolor='black')
    axes[2,1].set_title('Volume Ratio (Current/20-day MA)')
    axes[2,1].set_xlabel('Volume Ratio')
    
    plt.tight_layout()
    plt.show()
    
    # QQ plots for normality assessment
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    stats.probplot(df['daily_return'], dist="norm", plot=axes[0])
    axes[0].set_title('Q-Q Plot: Daily Returns vs Normal Distribution')
    
    stats.probplot(df['volatility_20'].dropna(), dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot: Volatility vs Normal Distribution')
    
    plt.tight_layout()
    plt.show()

## 4. Bivariate Relationships

In [ ]:
if not df.empty:
    # Relationship 1: Volume vs Returns
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Scatter plot: Volume Ratio vs Absolute Returns
    axes[0,0].scatter(df['volume_ratio'].dropna(), np.abs(df['daily_return']), alpha=0.5, s=10)
    axes[0,0].set_xlabel('Volume Ratio (Current/20-day MA)')
    axes[0,0].set_ylabel('Absolute Daily Return')
    axes[0,0].set_title('Volume Activity vs Return Magnitude')
    
    # Relationship 2: Volatility vs Returns
    vol_returns = df[['volatility_20', 'daily_return']].dropna()
    axes[0,1].scatter(vol_returns['volatility_20'], np.abs(vol_returns['daily_return']), alpha=0.5, s=10)
    axes[0,1].set_xlabel('20-Day Volatility')
    axes[0,1].set_ylabel('Absolute Daily Return')
    axes[0,1].set_title('Historical Volatility vs Current Return')
    
    # Time series: Price evolution by symbol
    for symbol in df['symbol'].unique()[:5]:  # Show top 5 symbols
        symbol_data = df[df['symbol'] == symbol].sort_values('date')
        axes[1,0].plot(symbol_data['date'], symbol_data['close'], label=symbol, alpha=0.8)
    axes[1,0].set_xlabel('Date')
    axes[1,0].set_ylabel('Close Price')
    axes[1,0].set_title('Price Evolution Over Time')
    axes[1,0].legend()
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Relationship 3: Price to SMA ratio vs Future Returns
    # Calculate next-day return for analysis
    df_temp = df.copy()
    df_temp['next_return'] = df_temp.groupby('symbol')['daily_return'].shift(-1)
    
    sma_data = df_temp[['price_to_sma20', 'next_return']].dropna()
    axes[1,1].scatter(sma_data['price_to_sma20'], sma_data['next_return'], alpha=0.5, s=10)
    axes[1,1].set_xlabel('Price to 20-day SMA Ratio')
    axes[1,1].set_ylabel('Next Day Return')
    axes[1,1].set_title('Technical Signal vs Future Performance')
    axes[1,1].axvline(1.0, color='red', linestyle='--', alpha=0.7, label='SMA Level')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()

## 5. Correlation Analysis

In [ ]:
if not df.empty:
    # Select key numeric variables for correlation
    corr_vars = ['close', 'volume', 'daily_return', 'volatility_20', 'price_range_pct', 
                 'volume_ratio', 'price_to_sma20']
    
    corr_data = df[corr_vars].dropna()
    correlation_matrix = corr_data.corr()
    
    # Correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    plt.title('Correlation Matrix: Key Financial Variables')
    plt.tight_layout()
    plt.show()
    
    print("🔗 Strongest Correlations:")
    # Find strongest correlations (excluding diagonal)
    corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            var1 = correlation_matrix.columns[i]
            var2 = correlation_matrix.columns[j]
            corr_val = correlation_matrix.iloc[i, j]
            corr_pairs.append((var1, var2, abs(corr_val), corr_val))
    
    # Sort by absolute correlation
    corr_pairs.sort(key=lambda x: x[2], reverse=True)
    
    for var1, var2, abs_corr, corr in corr_pairs[:5]:
        print(f"{var1} ↔ {var2}: {corr:.3f}")

## 6. Temporal and Seasonal Patterns

In [ ]:
if not df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Monthly seasonality
    monthly_returns = df.groupby('month')['daily_return'].agg(['mean', 'std']).reset_index()
    axes[0,0].bar(monthly_returns['month'], monthly_returns['mean'], 
                  yerr=monthly_returns['std'], capsize=5, alpha=0.7)
    axes[0,0].set_xlabel('Month')
    axes[0,0].set_ylabel('Average Daily Return')
    axes[0,0].set_title('Seasonal Pattern: Monthly Returns')
    axes[0,0].set_xticks(range(1, 13))
    
    # Day of week effect
    weekday_returns = df.groupby('weekday')['daily_return'].agg(['mean', 'std']).reset_index()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    axes[0,1].bar(weekday_returns['weekday'], weekday_returns['mean'],
                  yerr=weekday_returns['std'], capsize=5, alpha=0.7)
    axes[0,1].set_xlabel('Day of Week')
    axes[0,1].set_ylabel('Average Daily Return')
    axes[0,1].set_title('Day-of-Week Effect')
    axes[0,1].set_xticks(range(7))
    axes[0,1].set_xticklabels(weekday_names)
    
    # Quarterly patterns
    quarterly_vol = df.groupby('quarter')['volatility_20'].mean().reset_index()
    axes[1,0].bar(quarterly_vol['quarter'], quarterly_vol['volatility_20'], alpha=0.7)
    axes[1,0].set_xlabel('Quarter')
    axes[1,0].set_ylabel('Average Volatility')
    axes[1,0].set_title('Quarterly Volatility Patterns')
    
    # Rolling correlation over time (example: AAPL vs MSFT)
    if len(df['symbol'].unique()) >= 2:
        symbol1, symbol2 = df['symbol'].unique()[:2]
        
        # Create pivot table for correlation calculation
        returns_pivot = df.pivot_table(index='date', columns='symbol', values='daily_return')
        
        if symbol1 in returns_pivot.columns and symbol2 in returns_pivot.columns:
            rolling_corr = returns_pivot[symbol1].rolling(60).corr(returns_pivot[symbol2])
            
            axes[1,1].plot(rolling_corr.index, rolling_corr.values, alpha=0.8)
            axes[1,1].set_xlabel('Date')
            axes[1,1].set_ylabel('60-Day Rolling Correlation')
            axes[1,1].set_title(f'Rolling Correlation: {symbol1} vs {symbol2}')
            axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

## 7. Key Findings and Insights

### Top 3 Insights

**1. Return Distribution Characteristics**
- Daily returns exhibit significant departure from normality with fat tails and excess kurtosis
- This suggests higher probability of extreme events than normal distribution would predict
- **Implication**: Standard risk models assuming normality may underestimate tail risks

**2. Volume-Volatility Relationship**
- Strong positive correlation between volume activity and return magnitude
- High volume days tend to coincide with larger price movements (both positive and negative)
- **Implication**: Volume can serve as a leading indicator for volatility forecasting

**3. Temporal Patterns in Returns**
- Evidence of seasonal effects in both returns and volatility
- Day-of-week effects may indicate systematic behavioral biases
- **Implication**: Time-based features could improve predictive models

### Assumptions and Risks

**Key Assumptions:**
1. **Market Efficiency**: Price movements reflect all available information
2. **Stationarity**: Statistical relationships remain stable over time
3. **Independence**: Daily observations are independent (no autocorrelation)
4. **Data Quality**: API data is accurate and complete

**Identified Risks:**
1. **Regime Changes**: Market conditions can shift, invalidating historical patterns
2. **Survivorship Bias**: Analysis only includes currently active stocks
3. **Look-ahead Bias**: Risk of using future information in feature engineering
4. **Outlier Sensitivity**: Extreme events may disproportionately influence results

### Implications for Next Steps

**Feature Engineering Priorities:**
1. **Volatility Features**: Rolling volatility, GARCH-based measures
2. **Technical Indicators**: Moving averages, momentum indicators, RSI
3. **Volume Features**: Volume-price trend, volume breakouts
4. **Time Features**: Seasonal dummies, day-of-week indicators

**Data Preprocessing Needs:**
1. **Outlier Treatment**: Implement robust outlier detection and handling
2. **Normalization**: Consider log transforms for skewed variables
3. **Missing Data**: Develop strategy for handling gaps in time series
4. **Feature Scaling**: Standardize features for machine learning models

**Modeling Considerations:**
1. **Non-linear Models**: Consider tree-based models for capturing complex relationships
2. **Time Series Models**: Account for temporal dependencies in data
3. **Ensemble Methods**: Combine multiple models to improve robustness
4. **Risk Models**: Implement models that account for fat-tailed distributions

## 8. Summary and Next Steps

In [ ]:
print("\n🎯 EDA Summary:")
print("✅ Comprehensive statistical profiling completed")
print("✅ Distribution analysis reveals non-normal characteristics")
print("✅ Strong volume-volatility relationships identified")
print("✅ Temporal patterns and seasonality documented")
print("✅ Correlation structure mapped")

print("\n📊 Key Statistics:")
if not df.empty:
    print(f"Dataset size: {df.shape[0]:,} observations, {df.shape[1]} features")
    print(f"Time span: {(df['date'].max() - df['date'].min()).days} days")
    print(f"Average daily return: {df['daily_return'].mean():.4f}")
    print(f"Daily volatility: {df['daily_return'].std():.4f}")
    print(f"Return skewness: {df['daily_return'].skew():.4f}")
    print(f"Return kurtosis: {df['daily_return'].kurtosis():.4f}")

print("\n🔍 Ready for Feature Engineering:")
print("- Volatility-based features")
print("- Technical indicators")
print("- Volume-price relationships")
print("- Temporal/seasonal features")
print("- Risk-adjusted metrics")